In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.feature_selection import RFE
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [3]:
train = pd.read_csv('./Data/train.csv').drop(columns='id')
test = pd.read_csv('./Data/test.csv')

In [5]:
train.head()

,target,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,1.0,-1.067,-1.114,-0.616,0.376,1.090,0.467,-0.422,0.460,-0.443,...,0.220,-0.339,0.254,-0.179,0.352,0.125,0.347,0.436,0.958,-0.824
1,0.0,-0.831,0.271,1.716,1.096,1.731,-0.197,1.904,-0.265,0.557,...,-0.765,-0.735,-1.158,2.554,0.856,-1.506,0.462,-0.029,-1.932,-0.343
2,0.0,0.099,1.390,-0.732,-1.065,0.005,-0.081,-1.450,0.317,-0.624,...,-1.311,0.799,-1.001,1.544,0.575,-0.309,-0.339,-0.148,-0.646,0.725
3,1.0,-0.989,-0.916,-1.343,0.145,0.543,0.636,1.127,0.189,-0.118,...,-1.370,1.093,0.596,-0.589,-0.649,-0.163,-0.958,-1.081,0.805,3.401
4,0.0,0.811,-1.509,0.522,-0.360,-0.220,-0.959,0.334,-0.566,-0.656,...,-0.178,0.718,-1.017,1.249,-0.596,-0.445,1.751,1.442,-0.393,-0.643


In [22]:
test.head()

,id,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,250,-0.677,1.721,-0.745,-0.838,0.149,-1.138,0.242,0.504,-1.829,...,-1.180,-0.403,0.759,-0.600,0.951,-0.349,0.446,-0.819,-0.277,1.297
1,251,-0.731,-0.251,0.059,0.054,1.149,2.462,0.836,0.719,-2.269,...,1.297,1.114,0.657,0.760,0.899,-1.612,-1.701,1.107,-0.314,-0.641
2,252,1.119,1.036,1.220,1.518,0.265,-0.088,0.245,-0.533,-0.921,...,-0.864,-0.736,0.367,0.154,0.830,-1.352,0.914,0.377,0.588,-0.912
3,253,-0.933,0.212,-0.053,0.570,-1.540,-1.108,0.462,1.022,-0.215,...,0.063,-0.958,0.762,-0.213,-2.171,0.830,1.435,0.125,2.782,0.619
4,254,-0.208,-0.556,2.641,0.853,-0.384,0.312,0.514,0.481,-1.929,...,-0.689,0.213,0.568,-0.935,-0.015,0.267,0.739,1.340,-0.178,1.010


In [19]:
y_train = train['target']
X_train = train.drop('target', axis=1)
X_test = test.drop('id', axis = 1)

In [21]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.677,1.721,-0.745,-0.838,0.149,-1.138,0.242,0.504,-1.829,-1.380,...,-1.180,-0.403,0.759,-0.600,0.951,-0.349,0.446,-0.819,-0.277,1.297
1,-0.731,-0.251,0.059,0.054,1.149,2.462,0.836,0.719,-2.269,0.584,...,1.297,1.114,0.657,0.760,0.899,-1.612,-1.701,1.107,-0.314,-0.641
2,1.119,1.036,1.220,1.518,0.265,-0.088,0.245,-0.533,-0.921,0.714,...,-0.864,-0.736,0.367,0.154,0.830,-1.352,0.914,0.377,0.588,-0.912
3,-0.933,0.212,-0.053,0.570,-1.540,-1.108,0.462,1.022,-0.215,-0.205,...,0.063,-0.958,0.762,-0.213,-2.171,0.830,1.435,0.125,2.782,0.619
4,-0.208,-0.556,2.641,0.853,-0.384,0.312,0.514,0.481,-1.929,-0.400,...,-0.689,0.213,0.568,-0.935,-0.015,0.267,0.739,1.340,-0.178,1.010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19745,0.630,-0.244,-1.175,0.948,1.254,-1.297,-1.317,1.470,-1.958,0.543,...,-1.171,0.902,-0.006,-0.893,0.238,1.157,0.895,-1.760,-2.381,0.901
19746,-0.963,0.573,-0.482,-0.519,-0.433,-2.687,0.027,0.500,1.498,0.963,...,1.366,-0.262,1.583,2.449,-1.081,-0.199,-1.342,-1.732,0.263,-0.337
19747,-1.564,0.010,-1.097,0.434,-0.271,-1.777,0.184,-0.916,-0.527,1.307,...,-0.712,0.212,0.613,0.514,0.440,0.640,0.175,0.825,-0.809,1.653
19748,0.196,-3.164,0.903,0.638,0.615,0.295,0.580,0.398,-0.510,0.314,...,-0.406,1.489,1.571,-0.076,-0.438,0.755,1.812,0.770,-0.870,0.160


In [24]:
standard = StandardScaler()
X_train = standard.fit_transform(X_train)
X_test = standard.fit_transform(X_test)

In [35]:
xgb_model = xgb.XGBClassifier(random_state=42)
param_grid = {
    'objective':['binary:logistic'],
    'learning_rate': [0.001, 0.05, 0.1, 10],
    'max_depth':[2, 3, 4, 5, 6],
    'min_child_weight': [11],
    'subsample': [0.8],
    'colsample_bytree': [0.7],
    'n_estimators': [1000]
}
grid = GridSearchCV(estimator=xgb_model, cv=5, param_grid=param_grid, scoring='roc_auc', n_jobs=2, refit=True)
grid.fit(X_train, y_train)

print("Best Score:" + str(grid.best_score_))
print("Best Parameters:" + str(grid.best_params_))

best_parameters = grid.best_params_

Best Score:0.6350155100155099
Best Parameters:{'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 11, 'n_estimators': 1000, 'objective': 'binary:logistic', 'subsample': 0.8}


In [30]:
xgb_model = xgb.XGBClassifier(**best_parameters)
xgb_model.fit(X_train, y_train)

selector = RFE(estimator=xgb_model, n_features_to_select=100, step=1)
selector.fit(X_train, y_train)

xgb_preds = selector.predict_proba(X_test)[:,1]
train_predict = selector.predict(X_train)
roc_auc_score(y_train, train_predict)

0.5

In [32]:
# Using Grid Search to Find the Best Parameters of a Logical Regression Model
lr = LogisticRegression(random_state=42)
param_grid = {'class_weight' : ['balanced', None], 
              'penalty' : ['l2','l1'], 
              'C' : [0.001, 0.01, 0.1, 1, 10, 100],
              'solver': ['saga']}

grid = GridSearchCV(estimator = lr, cv=5, param_grid = param_grid , scoring = 'roc_auc', verbose = 1, n_jobs = -1)
grid.fit(X_train,y_train)

print("Best Score:" + str(grid.best_score_))
print("Best Parameters: " + str(grid.best_params_))

best_parameters = grid.best_params_

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best Score:0.7424652674652675
Best Parameters: {'C': 0.1, 'class_weight': None, 'penalty': 'l1', 'solver': 'saga'}


In [33]:
# Recursive feature elimination using RFE functions for logical regression models (select the first 150)
lr = LogisticRegression(**best_parameters)
lr.fit(X_train,y_train)

selector = RFE(estimator=lr, n_features_to_select=150, step=1)
selector.fit(X_train,y_train)

lr_preds = lr.predict_proba(X_test)[:,1]

train_predict = lr.predict(X_train)
roc_auc_score(y_train, train_predict)

0.7211483565777669

In [34]:
# Model fusion, with higher accuracy models given higher weights, and finally prediction
final_preds = (lr_preds * 0.8 + xgb_preds * 0.2)
submission = pd.read_csv('./Data/sample_submission.csv')
submission['target'] = final_preds
submission.to_csv('./Data/submission.csv', index=False)
submission.head(20)

,id,target
0,250,0.453331
1,251,0.254590
2,252,0.320030
3,253,0.364925
4,254,0.400505
5,255,0.507497
6,256,0.343448
7,257,0.142134
8,258,0.370999
9,259,0.553711
